In [46]:
#Nanisetty Sai Shailesh
#18IM30013

In [179]:
Pop_Size=10
Max_Iterations_GA=18
Init_Temp=950
Cooling_Factor=0.88
Same_Temp_Iter=8
Acceptance_Limit=2
Crossover_Probability=0.8
Mutation_Probability=0.006
import numpy as np
import random

In [180]:
Rl=[0.021,0.022,0.021,0.027,0.025,0.026,0.023,0.021,0.028,0.022]#Loan Interest rate
loss=[0.0002,0.0058,0.0001,0.0003,0.0024,0.0002,0.0058,0.0002,0.001,0.001]#Expected loan loss
rT=0.01#Customer Transaction rate
rD=0.009#Bank's deposit interest rate
D=60#Financial institution's deposit
K=0.15#Required reserve ratio
L=[10,25,4,11,18,3,17,15,9,10]#Loan size

In [181]:
Length=len(L) 
e=2.7182 #value of constant e

In [182]:
def Loan_revenue(Length,Rl,L,loss,Chromosome):
    nuval=0
    i=0
    while i<Length:
        nuval+=(Rl[i]*L[i]-loss[i])*Chromosome[i]
        i+=1
    return nuval

In [183]:
def Institutional_Transaction_Cost(K,D,L):
    T=(1-K)*D-L
    return T

In [184]:
def Total_Transaction_Cost(Length,rT,K,D,L,Chromosome):
    omegaval=0
    j=0
    while j<Length:
        omegaval+=rT*Institutional_Transaction_Cost(K,D,L[j])*Chromosome[j]
        j+=1
    return omegaval

In [185]:
def Demand_Deposit_Cost(rD,D):
    return rD*D

In [186]:
def Fx(Length,Rl,rT,rD,D,K,L,loss,Chromosome,flag=0):
    loss_term=0
    for k in range(Length):
        loss_term+=loss[k]*Chromosome[k]
    if flag==1:
        print(Loan_revenue(Length,Rl,L,loss,Chromosome),Total_Transaction_Cost(Length,rT,K,D,L,Chromosome),Demand_Deposit_Cost(rD,D),loss_term)
    return Loan_revenue(Length,Rl,L,loss,Chromosome)+Total_Transaction_Cost(Length,rT,K,D,L,Chromosome)-Demand_Deposit_Cost(rD,D)-loss_term

In [187]:
def Loss_Sum_Calculation(Length,Chromosome,L):
    LSum=0
    l=0
    while l<Length:
        LSum+=Chromosome[l]*L[l]
        l+=1
    return LSum

In [188]:
def CheckConstraint(Length,Chromosome,L,K,D):
    while(1):
        if Loss_Sum_Calculation(Length,Chromosome,L) > (1-K)*D:
            Chromosome=[]
            for j in range(Length):
                Chromosome.append(random.choice([0,1]))
        else:
            break
    return Chromosome

In [189]:
def ChromosomeCorrector(Length,Pop_Size,Population,L,K,D):
    i=0
    while i<Pop_Size:
        Population[i] = CheckConstraint(Length,Population[i],L,K,D)
        i+=1
    return Population

In [190]:
def InitialPopulation(Pop_Size,Length,L,K,D):
    Population=[]
    for i in range(Pop_Size):
        Chromosome=[]
        for j in range(Length):
            Chromosome.append(random.choice([0,1]))
        Population.append(Chromosome)
    return Population

In [191]:
def Fitness_value_calculation(Length,Rl,rT,rD,D,K,L,loss,Population,flag=0):
    Fitnessvals=[]
    for i in range(Pop_Size):
        Fitnessvals.append(Fx(Length,Rl,rT,rD,D,K,L,loss,Population[i],flag))
    return Fitnessvals

In [192]:
def Roulette(Pop_Size,Fitnessvals,Population):
    UpperLimits=[]
    Parents=[]
    Total_Fitness=sum(Fitnessvals)
    i=0
    while i<Pop_Size:
        if i==0:
            UpperLimits.append(Fitnessvals[i]/Total_Fitness)
        elif i==Pop_Size:
            UpperLimits.append(1) #Due to float values sometime the sum does not come equal to 1 but comes as 0.99999... so we assign it directly to 1 to avoid error in any case
        else:
            UpperLimits.append(UpperLimits[i-1]+Fitnessvals[i]/Total_Fitness)
        i+=1
    j=0
    k=0
    while j<Pop_Size:
        rand_num=random.random()
        while k<Pop_Size:
            if rand_num < UpperLimits[k]:
                Parents.append(Population[k])
                break
            k+=1
        j+=1    
    return Parents

In [193]:
#SPC=Single point type of crossover
def SPC(Length,Parent1,Parent2,Crossover_Probability):
    rand_num=random.random()
    if rand_num < Crossover_Probability:
        Child1=[]
        Child2=[]
        Crossover_Point=random.choice([i for i in range(1,Length)])
        z=0
        while z<Length:
            if z < Crossover_Point:
                Child1.append(Parent1[z])
                Child2.append(Parent2[z])
            else:
                Child1.append(Parent2[z])
                Child2.append(Parent1[z])
            z+=1    
        return Child1,Child2
    else:
        return Parent1,Parent2

In [194]:
def Crossover(Length,Pop_Size,Parents,Crossover_Probability):
    i=0
    if Pop_Size%2==0:
        while(i<Pop_Size):
            Parents[i],Parents[i+1]=SPC(Length,Parents[i],Parents[i+1],Crossover_Probability)
            i+=2
        return Parents
    else:
        while(i<Pop_Size-1):
            Parents[i],Parents[i+1]=SPC(Length,Parents[i],Parents[i+1],Crossover_Probability)
            i+=2
        return Parents

In [195]:
def Mutation(Length,Pop_Size,Parents,Mutation_Probability,L,K,D):
    for i in range(Pop_Size):
        for j in range(Length):
            rand_num=random.random()
            if rand_num < Mutation_Probability:
                Parents[i][j]=int(Parents[i][j]==0)
    return Parents

In [196]:
def GlobalBestUpdater(Population,Fitnessvals,GlobalBest,GlobalBestSol):
    Bestval=max(Fitnessvals)
    if Bestval > GlobalBest:
        GlobalBest=Bestval
        GlobalBestSol=Population[Fitnessvals.index(max(Fitnessvals))]
    return GlobalBest,GlobalBestSol

In [197]:
def Genetic_Algorithm(Length,Pop_Size,Max_Iterations_GA,Rl,rT,rD,D,K,L,loss,Population):
    GlobalBest=0
    GlobalBestSol=[]
    Population=ChromosomeCorrector(Length,Pop_Size,Population,L,K,D)
    for i in range(Max_Iterations_GA):
        Fitnessvals=Fitness_value_calculation(Length,Rl,rT,rD,D,K,L,loss,Population)
        GlobalBest,GlobalBestSol = GlobalBestUpdater(Population,Fitnessvals,GlobalBest,GlobalBestSol)
        Population=Roulette(Pop_Size,Fitnessvals,Population)
        Population=Crossover(Length,Pop_Size,Population,Crossover_Probability)
        Population=Mutation(Length,Pop_Size,Population,Mutation_Probability,L,K,D)
        Population=ChromosomeCorrector(Length,Pop_Size,Population,L,K,D)
    Fitnessvals=Fitness_value_calculation(Length,Rl,rT,rD,D,K,L,loss,Population)
    GlobalBest,GlobalBestSol = GlobalBestUpdater(Population,Fitnessvals,GlobalBest,GlobalBestSol)
    print(GlobalBest,GlobalBestSol,Population)
    return GlobalBest,GlobalBestSol

In [198]:
Genetic_Algorithm(Length,Pop_Size,Max_Iterations_GA,Rl,rT,rD,D,K,L,loss,InitialPopulation(Pop_Size,Length,L,K,D))

2.7230000000000003 [1, 0, 0, 1, 0, 1, 1, 0, 1, 0] [[0, 0, 1, 1, 0, 1, 1, 0, 1, 0], [0, 0, 1, 1, 0, 1, 1, 0, 1, 0], [0, 0, 1, 1, 0, 1, 1, 0, 1, 0], [0, 0, 1, 1, 0, 1, 1, 0, 1, 0], [0, 0, 1, 1, 0, 1, 1, 0, 1, 0], [0, 0, 1, 1, 0, 1, 1, 0, 1, 0], [0, 0, 1, 1, 0, 1, 1, 0, 1, 0], [0, 0, 1, 1, 0, 1, 1, 0, 1, 0], [0, 0, 1, 1, 0, 1, 1, 0, 1, 0], [0, 0, 1, 1, 0, 1, 1, 0, 1, 0]]


(2.7230000000000003, [1, 0, 0, 1, 0, 1, 1, 0, 1, 0])

In [199]:
def SA(Init_Temp,Cooling_Factor,Same_Temp_Iter,Acceptance_Limit,Length,Pop_Size,Max_Iterations_GA,Rl,rT,rD,D,K,L,loss):
    GlobalBest=0
    GlobalBestSol=[]
    Temp=Init_Temp
    Population=InitialPopulation(Pop_Size,Length,L,K,D)
    while Temp > 25:
        Acceptance_Counter=0
        for i in range(Same_Temp_Iter):
            Best,BestSol=Genetic_Algorithm(Length,Pop_Size,Max_Iterations_GA,Rl,rT,rD,D,K,L,loss,Population)
            rand_num=random.random()
            if Best>GlobalBest:
                GlobalBest=Best
                GlobalBestSol=BestSol
                #Population=Sample_Population
                Acceptance_Counter+=1
            elif rand_num < np.power(e,((GlobalBest-Best)/Temp)):
                #Population=Sample_Population
                Acceptance_Counter+=1
            if Acceptance_Counter==Acceptance_Limit:
                break
        Temp=Cooling_Factor*Temp
    return GlobalBest,GlobalBestSol 

In [202]:
SA(Init_Temp,Cooling_Factor,Same_Temp_Iter,Acceptance_Limit,Length,Pop_Size,Max_Iterations_GA,Rl,rT,rD,D,K,L,loss)
#Check the output at the bottom,i.e. the final best optimal solution

2.6362000000000005 [1, 0, 1, 0, 1, 1, 0, 0, 1, 0] [[0, 0, 1, 0, 0, 0, 0, 1, 1, 1], [0, 0, 1, 0, 0, 0, 0, 1, 1, 1], [0, 0, 1, 0, 0, 0, 0, 1, 1, 1], [0, 0, 1, 0, 0, 0, 0, 1, 1, 0], [0, 0, 1, 0, 0, 0, 0, 1, 1, 1], [0, 0, 1, 0, 0, 0, 0, 1, 1, 1], [0, 0, 1, 0, 0, 0, 0, 1, 1, 1], [0, 0, 1, 0, 0, 0, 0, 1, 1, 1], [0, 0, 1, 0, 0, 0, 0, 1, 1, 1], [0, 0, 1, 0, 0, 0, 0, 1, 1, 1]]
2.6362000000000005 [1, 0, 1, 0, 1, 1, 0, 0, 1, 0] [[1, 0, 1, 0, 0, 0, 0, 1, 0, 0], [1, 0, 1, 0, 0, 0, 0, 1, 0, 0], [1, 0, 1, 0, 0, 0, 0, 1, 0, 0], [1, 0, 1, 0, 0, 0, 0, 1, 0, 0], [1, 0, 1, 0, 0, 0, 0, 1, 0, 0], [1, 0, 1, 0, 0, 0, 0, 1, 0, 1], [0, 0, 1, 0, 0, 0, 0, 1, 0, 0], [1, 0, 1, 0, 0, 0, 0, 1, 0, 0], [1, 0, 1, 0, 0, 0, 0, 1, 0, 0], [1, 0, 1, 0, 0, 0, 0, 1, 0, 0]]
2.6362000000000005 [1, 0, 1, 0, 1, 1, 0, 0, 1, 0] [[1, 0, 0, 0, 0, 0, 1, 1, 0, 0], [0, 0, 0, 0, 1, 1, 1, 0, 0, 1], [0, 0, 0, 0, 1, 1, 1, 0, 0, 1], [0, 0, 0, 0, 1, 1, 1, 0, 0, 1], [0, 0, 0, 0, 1, 1, 1, 0, 0, 1], [0, 0, 0, 0, 1, 1, 1, 0, 0, 1], [0, 0, 0, 0, 1,

2.7066 [1, 0, 1, 0, 1, 0, 0, 0, 1, 1] [[1, 0, 1, 0, 0, 0, 0, 0, 1, 1], [0, 0, 1, 0, 0, 0, 0, 0, 1, 1], [1, 0, 1, 0, 0, 0, 0, 0, 1, 1], [1, 0, 1, 0, 0, 0, 0, 0, 1, 1], [1, 0, 1, 0, 0, 0, 0, 0, 1, 1], [1, 0, 1, 0, 0, 0, 0, 0, 1, 1], [1, 0, 1, 0, 0, 0, 0, 0, 1, 1], [1, 0, 1, 0, 0, 0, 0, 0, 1, 1], [1, 0, 1, 0, 0, 0, 0, 0, 1, 1], [1, 0, 1, 0, 0, 0, 0, 0, 1, 1]]
3.1854 [1, 0, 1, 1, 0, 1, 0, 0, 1, 1] [[1, 0, 1, 0, 1, 0, 0, 0, 1, 1], [1, 0, 1, 0, 1, 0, 0, 0, 1, 1], [1, 0, 1, 0, 1, 0, 0, 0, 1, 1], [1, 0, 1, 0, 1, 0, 0, 0, 1, 1], [1, 0, 1, 0, 1, 0, 0, 0, 1, 1], [1, 0, 1, 0, 1, 0, 0, 0, 1, 1], [1, 0, 1, 0, 1, 0, 0, 0, 1, 1], [1, 0, 1, 0, 1, 0, 0, 0, 1, 1], [1, 0, 1, 0, 1, 0, 0, 0, 1, 1], [1, 0, 1, 0, 1, 0, 0, 0, 1, 1]]
2.7066 [1, 0, 1, 0, 1, 0, 0, 0, 1, 1] [[1, 0, 0, 1, 0, 0, 0, 0, 1, 0], [1, 0, 0, 1, 0, 0, 1, 0, 1, 0], [1, 0, 0, 1, 0, 0, 0, 0, 1, 0], [1, 0, 0, 1, 0, 0, 0, 0, 1, 0], [1, 0, 0, 1, 0, 0, 0, 0, 1, 0], [1, 0, 0, 1, 0, 0, 0, 0, 1, 0], [1, 0, 0, 1, 0, 0, 0, 0, 1, 0], [1, 0, 0, 1, 0, 0, 

(3.1854, [1, 0, 1, 1, 0, 1, 0, 0, 1, 1])